# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime       # Import datetime function
from datetime import timedelta      # Import timedelta function

from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2   # Number of Bitcoins currently held
my_eth = 5.3   # Number of Etherium currently held

In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
# Fetch current BTC price
btc_data = requests.get(btc_url).json()
btc_price = btc_data["data"]["1"]["quotes"]["CAD"]["price"]

# Fetch current ETH price
eth_data = requests.get(eth_url).json()
eth_price = eth_data["data"]["1027"]["quotes"]["CAD"]["price"]

# Compute current value of my crpto
my_btc_value = btc_price * my_btc
my_eth_value = eth_price * my_eth

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0,.2f}")     # Show the current BTC holding with value formatted with the thousands separator and 2 decimal places
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0,.2f}")     # Show the current ETH holding with value formatted with the thousands separator and 2 decimal places

# Acknowledgement https://docs.python.org/3/library/string.html#formatspec
# Please also refer to the end for an alternate solution.

### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.environ.get('ALPACA_API_KEY')        # Fetch the api key from the .env file
alpaca_secret_key = os.environ.get('ALPACA_SECRET_KEY')  # Fetch the secret key from the .env file

# Set the Alpaca API version
alpaca_api_version = "v2"

# Set the base URL to Alpaca Market's trading environment.
# Could have read this from the .env file but Bootcamp's AlpacaMarkets_Installation-Guide.md in the 05-APIs/Supplemental folder setup instructions don't mention setting this up.
# According to https://github.com/alpacahq/alpaca-trade-api-python/#alpaca-environment-variables the default is the live site, so to be safe even though we are just fetchting prices, we
# hard code the base url to the paper trading site.
alpaca_base_url = "https://paper-api.alpaca.markets"     # Set the base url to Alpaca's paper trading environment

# Create the Alpaca API object
alpaca_api = tradeapi.REST(key_id=alpaca_api_key, secret_key=alpaca_secret_key, base_url=alpaca_base_url, api_version=alpaca_api_version)

In [ ]:
type(alpaca_api)

In [ ]:
# Format current date as ISO format
# today = pd.Timestamp.now(tz="America/New_York").isoformat()       # Get current date (see note below)

##################################################################################################
# NOTE TO ASSESSOR:
# Alpaca returned the following error when trying to retrieve the prices for the current date:
#    "APIError: your subscription does not permit querying data from the past 15 minutes"
#
# Despite the error message the delayed time frame specified in the alpaca_api.get_bars request
# needs to be at least 12 hours, not 15 minutes.
# To overcome the above error and service restrictions, I opted for 1 day old data.
##################################################################################################

# Calculate yesterday's date as of midnight by getting the current datetime, subtracting 1 day,
# normalising to midnight and then formatting into iso format
yesterday = (pd.Timestamp.now( tz="America/New_York") + timedelta(days=-1)).normalize().isoformat()
  
# Set the tickers
tickers = ['AGG', 'SPY']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# 4. Get the current closing prices for `SPY` and `AGG` using Alpaca's `get_bars()` function.
# Transform the function's response to a Pandas DataFrame and preview the data.

# Get current closing prices for SPY and AGG
df_portfolio = alpaca_api.get_bars(
    tickers,
    timeframe,
    start = yesterday,
    end = yesterday
).df

# Display sample data
df_portfolio

In [ ]:
# Note to Assessor 
The following instructions originally in the starter file seem to be inconsistent with steps 4 and 5 of the original README.md briefing.   
As such these have been ignored as there is no need to reorganise the data frame in order to fetch the current prices.

In [ ]:
# Reorganize the DataFrame
# Separate ticker data
#### SEE NOTE ABOVE  ####
# Concatenate the ticker DataFrames

In [ ]:
# 5. Pick the `SPY` and `AGG` close prices from the Alpaca's `get_bars()` DataFrame response and store them as Python variables.
# Pick AGG and SPY close prices
spy_close_price = float(df_portfolio[df_portfolio["symbol"] == "SPY"]["close"])
agg_close_price = float(df_portfolio[df_portfolio["symbol"] == "AGG"]["close"])

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# 6. Compute the value in dollars of the current amount of shares and print the results.
# Compute the current value of shares
my_spy_value = spy_close_price * my_spy
my_agg_value = agg_close_price * my_agg

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0,.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0,.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
# YOUR CODE HERE!

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# YOUR CODE HERE!

# Reorganize the DataFrame
# Separate ticker data
# YOUR CODE HERE!


# Concatenate the ticker DataFrames
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")

---

# Alternate solution using a dictionary for "Collect Crypto Prices Using the requests Library"

In [ ]:
# Alternate solution using a dictionary

# Define a dictionary for my portfolio
# the dictionary's schema is:
#    "integer": {              Unique index number E.g. 0
#        "name": "string",     Coin long name E.g. Bitcoin (as per alternative.me naming convention https://alternative.me/crypto/api/)
#        "symbol": "string",   Coin ticker symbol E.g. BTC
#        "coin_id": "integer", Coin's identifier on alternative.me. Eg. 1027 for etherium
#        "holding": "1.2"      Quantity of coin currently held in my portfolio
#    },

my_portfolio = {
    "0": {
        "name": "Bitcoin",
        "symbol": "BTC",
        "coin_id": 1,
        "holding": "1.2"
    },
    "1": {
        "name": "Ethereum",
        "symbol": "ETH",
        "coin_id": 1027,
        "holding": "5.3"
    }
}

preferred_currency_code = "CAD"   # Set the preferred currency code to fetch from alternative.me

In [ ]:
my_portfolio

In [ ]:
total_portfolio_value = 0 # initialise the total portfolio value

for coin in my_portfolio:
    # Get the coin details from my portfolio dictionary
    coin_name = my_portfolio[coin]["name"]
    coin_symbol = my_portfolio[coin]["symbol"]
    coin_id = str(my_portfolio[coin]["coin_id"])
    coin_holding = float(my_portfolio[coin]["holding"])

    url = f"https://api.alternative.me/v2/ticker/{coin_name}/?convert={preferred_currency_code}"  # build the url with the coin name and desired currency code
    coin_data = requests.get(url).json()  # fetch the pricing data
    
    coin_price = float(coin_data["data"][coin_id]["quotes"][preferred_currency_code]["price"])   # Convert the coin price into a float
    coin_value = coin_price * coin_holding     # Muliply the current coin price by our holding of the coin to get the value
    total_portfolio_value += coin_value        # Add the value of the current coin to the total portfolio value

    time_stamp = coin_data["metadata"]["timestamp"]    # Additional information - grab the timestamp from the feed data which is in Unix epoch in seconds (eg 1544129220 )  
    date_time = datetime.fromtimestamp(time_stamp)     # ...and convert it to a human readable format
    
    # Show the date and time when the coin price was applicable, the coin name, symbol and holding value based on how many coins we have in the portfolio.
    # ...formatting the value with the thousands separator and 2 decimal places  
    print(f"The current value (as at {date_time}) of your {coin_name} ({coin_symbol}) in {preferred_currency_code} is ${coin_value:0,.2f}")

print(f"The current total value of your portfolio in {preferred_currency_code} is ${total_portfolio_value:0,.2f}")    